<a href="https://colab.research.google.com/github/annesjyu/NLP2/blob/main/04_NLP2_RNN_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence Prediction

A sequence is an ordered collection of items. Traditional mechine learning algorithms assume data items to be independent and identically distributed (IID), but language task uses tokens which depend on the tokens before and after them. For example,

<`Two dogs`> <`are`> barking at the little boy.

In which the verb must agree with the number of subjects.

Sometimes there dependencies can be arbitrarily long. For example,

The <`dog`> has a short, yet luxuriously puffy <`tail`>.

## Hidden State

It is a special intermediate vector generated from the previous tokens, which carries the context information seen so far. It can be used with the current input token to predict the next token.

The image below shows a character-level model taking hidden states together with input characters.

<img src="https://raw.githubusercontent.com/annesjyu/NLP2/fc8c2ed7d01030820709417e4a879ca4738fd687/rnn-train.svg" width=450></img>

<small> *Image credit to [ Recurrent Neural Networks. d2i.ai](https://d2l.ai/chapter_recurrent-neural-networks/rnn.html).</small>

## Recurrent Neural Networks

RNNs are a family of models following a recurrent architecture, which takes hidden states as well as current input to predict the next output. Here, we are going to introduct the simplest RNN, Elman RNN.

### Elman RNN

It is the RNN introducing the concept of internal memory into neural networks. The memory stores internal states based on previous inputs. It is designed to process sequences of data, for example, time series, sentences.

It is composed of 3 layers:

1. Input layer, is the input sequence.
2. `Hidden layer`, processes the inputs by taking hidden state computed from previous input and hidden state at that old time point. The recurrence forms the memory aspect of the network.
3. Output layer, produces the final output.

The network is also known as `backpropagation through time (BPTT)`, which is a variant of the standard backpropagation algorithm used to train RNNs by adapting to handle the `temporal` dimension.

<img src="https://github.com/annesjyu/NLP2/blob/main/rnn-2.gif?raw=true" width=350></img>
<small>* Loye, G. (2023, January 23). Beginner’s Guide on Recurrent Neural Networks with PyTorch. FloydHub Blog. https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/.</small>

### Mathematical Representation

Given an input of sequence items, $x_1, x_2, ..., x_T$, for each time step $t$, the hidden state $h_t$, and the output $y_t$ are computed as,

$
h_t = f(U*x_t + W * h_{t-1} + b), \\
y_t = V * h_t + c
$

Where,
* $f$ is an activation function, e.g. ReLU.
* $U$, $W$, and $V$ are the weight matrices for input, hidden, and output.
* $b$ and $c$ are bias vectors.
* $h_{t-1}$ is the hidden state from the previous time step.


### Implementation

In [ ]:
import torch.nn as nn

In [ ]:
class SimpleRNN(nn.Module):
  def __init__(self,
               input_size,
               hidden_size,
               output_size,
               num_layers=1):
    '''
    input_size: The number of expected features in the input x
    hidden_size: The number of features in the hidden state h
    output_size: The number of features in the output y
    num_layers: The number of recurrent layers.
    '''
    super(SimpleRNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    # The RNN layer.
    # batch_first: If True, then the input and output tensors are provided as
    #  (batch, seq, feature), instead of (seq, batch, feature). Note that this
    #  does not apply to hidden states or cell states. Default: False.
    self.rnn = nn.RNN(input_size=input_size,
                      hidden_size=hidden_size,
                      num_layers=num_layers,
                      batch_first=True, # the first batch
                      bias=True,
                      nonlinearity='tanh') # can also use relu
    # Go through another linear layer to compute final output.
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    # Initialize the hidden state with zeros.
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    hidden, _ = self.rnn(x, h0)
    # Pass the output of the last time step to the final classifier
    return self.fc(hidden[:, -1, :])

#### Training the simple RNN

In [ ]:
import torch
from torch import randn
import torch.optim as optim

In [ ]:
one_hot_size = 10
sequence_width = 7
num_classes = 3

model1 = SimpleRNN(input_size=one_hot_size,
                   hidden_size=32,
                   output_size=num_classes,
                   num_layers=2)
print(model1)

In [ ]:
def TrainRandn(model,
               n_epochs=5,
               n_batches=128,
               batch_size=3,
               sequence_width=7,
               one_hot_size=10,
               num_classes=3):
  '''
    Train the model with the given parameters using randomly generated data. The
    total size of input is batch_size * n_batches, with each input having a dim
    of sequence_width * one_hot_size.
  '''
  loss_fn = nn.MSELoss()
  optimizer = optim.SGD(model.parameters(), lr=0.001)

  for epoch in range(n_epochs):
    epoch_loss = 0
    for batch in range(n_batches):
        # Generate fake data for the batch.
        x = torch.randn(batch_size, sequence_width, one_hot_size)
        y_target = torch.randn(batch_size, num_classes)
        #print(f'x={x}')
        #print(f'y_target={y_target}')

        # Reset gradients
        optimizer.zero_grad()

        # Forward pass: compute predicted y by passing x to the model.
        y_pred = model(x)
        #print(f'y_pred={y_pred}')
        #print(f'y_pred.shape={y_pred.shape}')
        #print(f'y_target.shape={y_target.shape}')

        # Compute and print loss.
        loss = loss_fn(y_pred, y_target)
        batch_loss = loss.item()/batch_size
        epoch_loss += batch_loss

        # Propagate the loss value backward.
        loss.backward()

        # Trigger the optimizer to update once.
        optimizer.step()
        # End of one epoch

    epoch_loss /= n_batches
    print(f'epoch={epoch}, epoch_loss={epoch_loss}\n')

In [ ]:
TrainRandn(model1)

### Model Optimization

The problem of Elman RNN is its failing to capture long-range dependencies. If previous hidden state from previous tokens is irrelevant to the current state, it should not be included into the current output calculation. But Elman RNN doesn't control over hidden states. An enhanced architecture is called `Gated RNN` which selects hidden states to be included into calculation, therefore shows better performance on long-range language modeling tasks.

### Gated RNN

A `Gated Recurrent Neural Network (Gated RNN)` is a type of recurrent neural network that uses gating mechanisms to control the flow of hidden states. These gates effectively regulate the addition or removal of hidden states in the cell state, making the network capable of learning long-term dependencies in sequence data. Gated RNNs are particularly useful in addressing the `vanishing gradient` problem that is common in vanilla RNNs.

Types of Gated RNNs

1. Long Short-Term Memory (LSTM)

LSTM units include input, output, and forget gates. These gates determine what information should be retained or discarded at each step in the sequence, allowing the network to maintain a longer memory.

* Input Gate

> It decides which values from the input should be used to update the memory.

<img src="https://github.com/annesjyu/NLP2/blob/main/lstm_input_gate.gif?raw=true" width=350></img>

* Forget Gate

> It decides what information should be thrown away from the block.

<img src="https://github.com/annesjyu/NLP2/blob/main/lstm_forget_gate.gif?raw=true" width=350></img>

* Cell State

> It is a combination of input and forgot gate outputs. The current cell state gets pointwise multiplied by the forget vector. This has a possibility of dropping values in the cell state if it gets multiplied by values near 0. It then gets pointwise added with input gate vector to get a new cell state.

<img src="https://github.com/annesjyu/NLP2/blob/main/lstm_cell_state.gif?raw=true" width=350></img>

* Output Gate

> It determines what the next hidden state should be.

<img src="https://github.com/annesjyu/NLP2/blob/main/lstm_output_gate.gif?raw=true" width=350></img>

2. Gated Recurrent Unit (GRU)

GRUs are a simpler variant of LSTMs. They combine the input and forget gates into a single "update gate" and also merge the cell state and hidden state, resulting in a more streamlined model that can perform on par with LSTM for certain tasks.

* Update Gate

> It helps the model decide how much of the past information (from previous time steps) needs to be passed along to the future.

* Reset Gate

> It decides how much of the past information to forget.

<img src="https://raw.githubusercontent.com/annesjyu/NLP2/main/lstm_gated_rnn.webp" width=350></img>

#### Reference:

Phi, M. (2020, June 28). Illustrated Guide to LSTM’s and GRU’s: A step by step explanation. Medium. https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

#### Mathematical Representation

##### Gated RNN

$
h_t = h_{t-1} + λ(h_{t-1}, x_t) × F(U*x_t + W * h_{t-1} + b), \\
y_t = V * h_t + c
$

Where,

$λ(h_{t-1}, x_t)$ is the gating function, with a value in $[0,1]$. Furthur, $λ$ is context-dependent and it is usually a sigmoid function.

##### LSTM

Hochreiter and Schmidhuber describe the function as below,

$
h_t = μ(h_{t-1}, x_t)h_{t-1} + λ(h_{t-1}, x_t)F(h_{t-1}, x_t)
$

Where,

$μ$ is another gating function.

###### Reference:

* Hochreiter, S., Schmidhuber, J. "Long Short-Term Memory". Neural Comput 1997; 9 (8): 1735–1780. doi: https://doi.org/10.1162/neco.1997.9.8.1735

* Olah, C. "Understanding LSTM Networks". (August, 2015). https://colah.github.io/posts/2015-08-Understanding-LSTMs/



### Implementation

In [ ]:
class GatedRNN(nn.Module):
  def __init__(self,
               input_size,
               hidden_size,
               output_size,
               num_layers=1):
    '''
    input_size: The number of expected features in the input x
    hidden_size: The number of features in the hidden state h
    output_size: The number of features in the output y
    num_layers: The number of recurrent layers.
    '''
    super(SimpleRNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    # Gated Recurrent Network layer
    self.gru = nn.GRU(input_size=self.input_size,
                      hidden_size=self.hidden_size,
                      num_layers=self.num_layers,
                      batch_first=True) # it is true at the step of initialization
    # Go through another linear layer to compute final output.
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    # Initialize the hidden state with zeros.
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    hidden, _ = self.gru(x, h0)
    # Pass the output of the last time step to the final classifier
    return self.fc(hidden[:, -1, :])

#### Excercise

In [ ]:
# Can you train random data on a GatedRNN model
